## Capstone 1 - Part 2

In the first part of the capstone, we focused on Data Retrieval, Data Preprocessing, Feature Engineering and Exploratory Data Analysis using Python & its libraries. Now we are going to shift gears and gain insights into our HR Analytics dataset using SQL.

## TODO: Make use of SQL to do the following:

### Create a SQLITE3 DB using the CSV file (2 pts). Please refer this [link](https://www.linkedin.com/pulse/accessing-sqlite3-database-from-jupyter-notebook-using-varun-lobo/) and this [link](https://www.geeksforgeeks.org/how-to-import-csv-file-in-sqlite-database-using-python/) to know more.

### Calculate the Attrition Rate and summarize attrition (3 pts) by:
- Gender
- Department
- Age
- Average monthly income by job level
- Years at company

### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

### Effective Communication (2 pts)
- Please make use of markdown cells to communicate your thought process, why did you think of performing a step? what was the observation from the query? etc.
- The code should be commented so that it is readable for the reviewer.

### Grading and Important Instructions
- Each of the above steps are mandatory and should be completed in good faith
- Make sure before submitting that the code is in fully working condition
- It is fine to make use of ChatGPT, stackoverflow type resources, just provide the reference links from where you got it
- Debugging is an art, if you find yourself stuck with errors, take help of stackoverflow and ChatGPT to resolve the issue and if it's still unresolved, reach out to me for help.
- You need to score atleast 7/10 to pass the project, anything less than that will be marked required, needing resubmission.
- Feedback will be provided on 3 levels (Awesome, Suggestion, & Required). Required changes are mandatory to be made.
- For submission, please upload the project on github and share the link to the file with us through LMS.

In [1]:
import sqlite3
import pandas as pd

In [3]:
df=pd.read_csv('HR-Analytics.csv')
conn=sqlite3.connect('mytable.db')
df.to_sql(name="employee",con=conn,if_exists='replace',index=False)
conn.commit()
%load_ext sql
%sql sqlite:///mytable.db

#### Calculate Attrition Rate


In [5]:
### Implement code

attrition_rate = """
select case when Attrition = 'Yes' then 'true' else 'false' end as Attrition, 
round(count(*) * 100.0 / (select count(*) from employee), 1) || '%' as Attrition_rate
from employee 
group by Attrition
order by attrition desc;
"""
pd.read_sql(attrition_rate, conn)

,Attrition,Attrition_rate
0,true,16.1%
1,false,83.9%


##### From the above table we see that 16.1% of employees left the company

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857886?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NiwiaG1hYyI6ImUyNGU0YWRhNWQwMDkxNWE2NWNiY2EwNGFlNDNiMTBlYmYyNWY0YzAiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Gender

In [7]:
### Implement code

gender_attrition = """
select case when Attrition = 'Yes' then 'true' else 'false' end as Attrition,
Gender, 
count(*) as Count_gender,
round((count(*) * 100.0 / sum(count(*)) over (partition by Gender)), 1) as Attrition_by_gender
from employee
group by Attrition, Gender
order by Attrition desc;
"""
pd.read_sql(gender_attrition, conn)

,Attrition,Gender,Count_gender,Attrition_by_gender
0,true,Female,87,14.8
1,true,Male,150,17.0
2,false,Female,501,85.2
3,false,Male,732,83.0


##### Males have a higher attrition rate than females 

#### Sample output

![image](https://api-v4.skyprepapp.com/public_api/da/857882?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MiwiaG1hYyI6IjM1NDU5N2E4MjZmMTMxNWFhMTA4NzFiYTFkZjQ5YjBhNTYyOWYwMzgiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Dept

In [9]:
### Implement code

department_attrition = """
select Department, Attrition, count(*) as Department_attrition from employee
group by Department, Attrition
order by Department, Attrition;
"""
pd.read_sql(department_attrition, conn)

,Department,Attrition,Department_attrition
0,Human Resources,No,51
1,Human Resources,Yes,12
2,Research & Development,No,828
3,Research & Development,Yes,133
4,Sales,No,354
5,Sales,Yes,92


##### Research & Development has the highest attrition

#### Sample Output

<img src="https://api-v4.skyprepapp.com/public_api/da/857884?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NCwiaG1hYyI6IjdmOTA4NzJjODZmYTgxOGUyMzkyYTBlZjhjYjljNjA4ZGM3NjkzMWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9" width="450" height="450">

#### Find Attrition by Age Groups

In [11]:
### Implement code

age_attrition = """
select case when Age < 30 then 'Under 30'
when Age between 30 and 40 then '30 - 40'
when Age between 40 and 50 then '40-50'
else 'Over 50' end as Age_group,
case when Attrition = 'Yes' then 'true' else 'false' end as Attrition,
count(*) as num,
round((count(*) * 100.0 / sum(count(*)) over (partition by case 
when Age < 30 then 'Under 30'
when Age between 30 and 40 then '30 - 40'
when Age between 40 and 50 then '40-50'
else 'Over 50' end)), 2) as percent_by_age
from employee group by case 
when Age < 30 then 'Under 30'
when Age between 30 and 40 then '30 - 40'
when Age between 40 and 50 then '40-50'
else 'Over 50' end, 
Attrition
order by Attrition;
"""
pd.read_sql(age_attrition, conn)

,Age_group,Attrition,num,percent_by_age
0,30 - 40,false,585,86.16
1,40-50,false,288,89.44
2,Over 50,false,125,87.41
3,Under 30,false,235,72.09
4,30 - 40,true,94,13.84
5,40-50,true,34,10.56
6,Over 50,true,18,12.59
7,Under 30,true,91,27.91


##### Younger employees who are under 30 have the highest attrition rate, suggesting they may leave due to career growth opportunities, job satisfaction, or work conditions.

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857885?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NSwiaG1hYyI6IjcwNGVlOWIwYzg1MmMwNmNhODg0NmYyZDFlNWE3OTU1MTFhMGVmYWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Monthly Income

In [13]:
### Implement code

job_level_income = """
select a.Department, a.JobLevel, 
round(avg(a.MonthlyIncome), 1) as avg_income,
round(avg(b.MonthlyIncome), 1) as attrition_avg_income,
round(avg(b.MonthlyIncome) - avg(a.MonthlyIncome), 1) as difference
from employee a join employee b on a.Department = b.Department and a.JobLevel = b.JobLevel and b.Attrition = 'Yes'
group by a.Department, a.JobLevel
order by a.Department, a.JobLevel;
"""
pd.read_sql(job_level_income, conn)

,Department,JobLevel,avg_income,attrition_avg_income,difference
0,Human Resources,1,2733.2,2415.7,-317.5
1,Human Resources,3,9623.0,10216.0,593.0
2,Research & Development,1,2840.1,2687.4,-152.7
3,Research & Development,2,5291.2,5372.0,80.8
4,Research & Development,3,10170.5,9503.8,-666.6
5,Research & Development,4,15634.7,12169.0,-3465.7
6,Research & Development,5,19218.5,19550.0,331.5
7,Sales,1,2506.7,2373.4,-133.3
8,Sales,2,5746.1,5917.0,170.9
9,Sales,3,9282.3,9202.8,-79.5


##### Research & Development and Sales saw the biggest pay differences, suggesting dissatisfaction despite high salaries.

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857883?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MywiaG1hYyI6Ijg3NTU4ZDU1ZjRjN2U1YWI3ODQzYjM1NzFkNjBjMjEwNGY5NWI5ODUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Years At Company

In [15]:
### Implement code

years_at_company_attrition = """
select case 
when YearsAtCompany = 0 then 'New Hires'
when YearsAtCompany between 2 and 5 then '2-5 years'
when YearsAtCompany between 6 and 10 then '6-10 years'
when YearsAtCompany between 11 and 20 then '11-20 years'
else 'Over 20 years' end as tenure_years,
count(*) as num,
round((count(*) * 100.0 / sum(count(*)) over ()), 1) as percent
from employee
where Attrition = 'Yes'
group by tenure_years
order by num desc;
"""
pd.read_sql(years_at_company_attrition, conn)

,tenure_years,num,percent
0,2-5 years,87,36.7
1,Over 20 years,67,28.3
2,6-10 years,55,23.2
3,New Hires,16,6.8
4,11-20 years,12,5.1


##### Employees with 2-5 years of tenure have the highest attrition 

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857881?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MSwiaG1hYyI6IjFhYmY3NGI4MzQ1NzViMWZkNDJlMjcwYTUyOTQ0OWQwZjJjMjhmNWUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

In [17]:
over_50_vs_40_50 = """
select case 
when Age between 40 and 50 then '40-50'
when Age > 50 then 'Over 50' end as Age_Group,
JobRole,
round(avg(JobSatisfaction), 1) as Avg_Job_Satisfaction,
round(avg(WorkLifeBalance), 1) as Avg_Work_Life_Balance,
count(*) as Attrition_Count
from employee
where Attrition = 'Yes' and Age >= 40
group by Age_Group, JobRole
order by Age_Group, Attrition_Count desc;
"""
pd.read_sql(over_50_vs_40_50, conn)

,Age_Group,JobRole,Avg_Job_Satisfaction,Avg_Work_Life_Balance,Attrition_Count
0,40-50,Sales Executive,2.7,2.7,18
1,40-50,Laboratory Technician,2.0,2.1,7
2,40-50,Research Scientist,2.4,2.4,5
3,40-50,Sales Representative,2.7,2.7,3
4,40-50,Manager,2.0,2.5,2
5,40-50,Healthcare Representative,3.0,3.5,2
6,40-50,Research Director,3.0,3.0,1
7,40-50,Human Resources,1.0,3.0,1
8,Over 50,Sales Executive,2.5,2.0,4
9,Over 50,Research Scientist,2.5,3.0,4


##### Employees aged 40-50 in Laboratory Technician and Manager roles have the lowest job satisfaction and poor work-life balance, which may be key factors driving attrition in this age group.

In [19]:
higher_pay = """
select JobLevel,
round(avg(MonthlyIncome), 1) as Avg_Income,
round(avg(JobSatisfaction), 1) as Avg_Job_Satisfaction,
round(avg(WorkLifeBalance), 1) as Avg_Work_Life_Balance,
sum(case when OverTime = 'Yes' then 1 else 0 end) as OverTime_Count,
count(*) as Attrition_Count
from employee
where Attrition = 'Yes' and MonthlyIncome > (select avg(MonthlyIncome) from employee)
group by JobLevel
order by Avg_Income desc;
"""
pd.read_sql(higher_pay, conn)

,JobLevel,Avg_Income,Avg_Job_Satisfaction,Avg_Work_Life_Balance,OverTime_Count,Attrition_Count
0,5,19463.8,2.4,3.0,3,5
1,4,13150.4,2.6,2.0,3,5
2,3,9517.7,2.5,2.7,12,31
3,2,8146.2,1.9,2.9,5,11


##### Employees at Job Level 2 have the lowest job satisfaction, which along with moderate work-life balance and overtime, may be a key factor driving attrition in this group.

In [21]:
less_than_5 = """
select Department,
round(avg(JobSatisfaction), 1) as Avg_Job_Satisfaction,
round(avg(DistanceFromHome), 1) as Avg_Distance_From_Home,
sum(case when OverTime = 'Yes' then 1 else 0 end) as OverTime_Count,
count(*) as Attrition_Count
from employee
where Attrition = 'Yes' and YearsAtCompany < 5
group by Department
order by Attrition_Count desc;
"""
pd.read_sql(less_than_5, conn)

,Department,Avg_Job_Satisfaction,Avg_Distance_From_Home,OverTime_Count,Attrition_Count
0,Research & Development,2.5,10.3,45,79
1,Sales,2.4,10.7,27,53
2,Human Resources,2.2,13.7,3,9


##### Research & Development has the lowest average job satisfaction and the highest attrition count, while Human Resources has the lowest overtime count and the highest average distance from home.

In [23]:
job_satisfaction = """    
select JobRole, round(avg(JobSatisfaction), 1) as Avg_Job_Satisfaction,
count(*) as Attrition_Count
from employee
where Attrition = 'Yes'
group by JobRole
order by Avg_Job_Satisfaction;
"""
pd.read_sql(job_satisfaction, conn)

,JobRole,Avg_Job_Satisfaction,Attrition_Count
0,Human Resources,2.2,12
1,Laboratory Technician,2.4,62
2,Manager,2.4,5
3,Research Scientist,2.4,47
4,Research Director,2.5,2
5,Sales Executive,2.5,57
6,Sales Representative,2.5,33
7,Manufacturing Director,2.6,10
8,Healthcare Representative,2.8,9


##### Laboratory Technicians have the lowest average job satisfaction and the highest attrition count, while Healthcare Representatives have the highest job satisfaction and the lowest attrition count.


In [25]:
overtime = """
select OverTime, count(*) as Attrition_Count,
round((count(*) * 100.0 / (select count(*) from employee where Attrition = 'Yes')), 1) as Attrition_Percentage
from employee
where Attrition = 'Yes'
group by OverTime;
"""
pd.read_sql(overtime, conn)

,OverTime,Attrition_Count,Attrition_Percentage
0,No,110,46.4
1,Yes,127,53.6


##### Employees who worked overtime have a higher attrition count and percentage compared to those who did not work overtime

In [27]:
worklife_balance = """
select WorkLifeBalance, count(*) as Attrition_Count,
round((count(*) * 100.0 / (select count(*) from employee where Attrition = 'Yes')), 1) as Attrition_Percentage
from employee
where Attrition = 'Yes'
group by WorkLifeBalance
order by WorkLifeBalance;
"""
pd.read_sql(worklife_balance, conn)

,WorkLifeBalance,Attrition_Count,Attrition_Percentage
0,1,25,10.5
1,2,58,24.5
2,3,127,53.6
3,4,27,11.4


##### Employees with a WorkLifeBalance rating of 3 have the highest attrition count and percentage, while those with a rating of 1 have the lowest attrition count and percentage

In [29]:
commute = """
select case when DistanceFromHome <= 5 then '0-5 km'
when DistanceFromHome between 6 and 15 then '6-15 km'
when DistanceFromHome between 16 and 30 then '16-30 km'
else 'Over 30 km' end as Commute_Distance,
count(*) as Attrition_Count,
round((count(*) * 100.0 / (select count(*) from employee where Attrition = 'Yes')), 1) as Attrition_Percentage
from employee
where Attrition = 'Yes'
group by Commute_Distance
order by Attrition_Percentage desc;
"""
pd.read_sql(commute, conn)

,Commute_Distance,Attrition_Count,Attrition_Percentage
0,0-5 km,87,36.7
1,6-15 km,82,34.6
2,16-30 km,68,28.7


##### Employees with a commute distance of 0-5 km have the highest attrition count and percentage, while those with a commute distance of 16-30 km have the lowest attrition count and percentage 

In [31]:
years_in_current_role = """
select case when YearsInCurrentRole < 2 then 'Less than 2 years'
when YearsInCurrentRole between 2 and 5 then '2-5 years'
else 'Over 5 years' end as Role_Tenure,
count(*) as Attrition_Count,
round((count(*) * 100.0 / (select count(*) from employee where Attrition = 'Yes')), 1) as Attrition_Percentage
from employee
where Attrition = 'Yes'
group by Role_Tenure
order by Attrition_Percentage desc;
"""
pd.read_sql(years_in_current_role, conn)

,Role_Tenure,Attrition_Count,Attrition_Percentage
0,2-5 years,100,42.2
1,Less than 2 years,84,35.4
2,Over 5 years,53,22.4


##### Employees with a role tenure of 2-5 years have the highest attrition count and percentage, while those with a tenure of over 5 years have the lowest attrition count and percentage 

### Overall Conclusion
###### The data analysis reveals several key insights into employee attrition patterns within the company. A significant portion (16.1%) of employees left the company, with males showing a higher attrition rate than females. 
###### Research & Development department stands out with the highest attrition, possibly due to factors like job dissatisfaction, despite offering competitive pay. 
###### Younger employees, particularly those under 30, have the highest attrition, likely driven by career growth opportunities and job satisfaction concerns. 
###### Employees with 2-5 years of tenure are most likely to leave, pointing to potential challenges in career progression or engagement. 

###### Furthermore, certain employee groups, including those aged 40-50 in roles like Laboratory Technician and Manager, report low job satisfaction and poor work-life balance, which may be contributing to their higher attrition. 
###### Job Level 2 employees also exhibit low satisfaction, with the combination of moderate work-life balance and overtime possibly contributing to their departure. 
###### Addressing these factors—such as improving job satisfaction, work-life balance, and career growth opportunities—could help reduce attrition across the organization.